In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def get_page_content(request_url): 
    # 得到页面的内容
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
    html=requests.get(request_url, headers=headers, timeout=10)
    content = html.text
    # 通过content创建BeautifulSoup对象
    soup = BeautifulSoup(content, 'html.parser', from_encoding='utf-8')
    return soup

# 分析当前页面的投诉信息
def analysis(soup):
    df = pd.DataFrame(columns = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime', 'status'])
    # 找到完整的投诉信息框
    temp = soup.find('div',class_='tslb_b')
    # 找出所有的tr，即行
    tr_list = temp.find_all('tr')
    for tr in tr_list:
        td_list = tr.find_all('td')
        # 如果没有td，就是表头th
        if len(td_list) > 0:
            # 投诉编号，投诉品牌，投诉车系，投诉车型，问题简述，典型问题，投诉时间，投诉状态
            id, brand, car_model, type, desc, problem, datetime, status = \
                td_list[0].text, td_list[1].text, td_list[2].text, td_list[3].text, \
                td_list[4].text, td_list[5].text, td_list[6].text, td_list[7].text
            print(id, brand, car_model, type, desc, problem, datetime, status)
            temp = {}
            temp['id'] = id
            temp['brand'] = brand
            temp['car_model'] = car_model
            temp['type'] = type
            temp['desc'] = desc
            temp['problem'] = problem
            temp['datetime'] = datetime
            temp['status'] = status
            df = df.append(temp, ignore_index = True)
    return df
            
result = pd.DataFrame(columns = ['id', 'brand', 'car_model', 'type', 'desc', 'problem', 'datetime', 'status'])

# 请求URL
base_url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-0-0-0-0-0-' #1.shtml
page_num = 20
for i in range(page_num):
    # 拼接当前的页面URL
    request_url = base_url + str(i+1) + '.shtml'
    # 得到soup解析
    soup = get_page_content(request_url)
    # 通过soup解析，得到当前页面的dataframe
    df = analysis(soup)
    result = result.append(df)
    
print(result)
result.to_csv('car_complain.csv', index = False)
result.to_excel('car_complain.xlsx', index = False)


D:\Users\zhangwensheng\Anaconda3\lib\site-packages\bs4\__init__.py:203: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


        id    brand car_model                              type  \
0   576673     吉利汽车        嘉际                2019款 1.8TD 自动 悦享型   
1   576672  上汽大众斯柯达        速派           2018款 TSI280 DSG 舒适版 国V   
2   576671     吉利汽车        缤越          2019款 260T DCT 游侠 运动款 国V   
3   576669    一汽-大众        迈腾          2012款 2.0TSI DSG 尊贵型（改款）   
4   576663   东风悦达起亚        智跑               2018款 2.0L 自动 智享豪华版   
..     ...      ...       ...                               ...   
25  575335       星途     星途TXL                 2021款 1.6T 两驱 星耀版   
26  575334     长安福特        锐界      2020款 EcoBoost 245 两驱 豪锐型 7座   
27  575327     广汽丰田        雷凌  2017款 1.8H GS CVT 精英天窗版 国V 改款 双擎   
28  575325     东风日产        逍客                2016款 2.0L CVT 精英版   
29  575323  一汽-大众奥迪     奥迪A4L                 2016款 1.8T 自动 舒适型   

                    desc     problem    datetime status  
0     吉利嘉际4S店未返还购车时的旅游基金       L313,  2021-03-05   信息审核  
1       斯柯达速派变速箱故障灯亮挂挡不走   B96,B334,  2021-03-05   信息审核  
2   吉利缤越变速箱升挡时方向盘有抖动现